### Custom Prompt
示例：根据函数名称，查找函数代码，并给中文的代码说明

In [7]:
from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数作为示例
def hello_world(say):
    print("Hello World!")
    return say

PROMPT = """
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码，中文解释。
函数名称: {function_name}
源代码: {source_code}
代码解释:
"""

import inspect
def get_source_code(function_name):
    # 获得源代码
    return inspect.getsource(function_name)

# 自定义模板class
class CustomPrompt(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # Get source code
        source_code = get_source_code(kwargs["function_name"])
        # Generate promopt
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt
    
prompt_obj = CustomPrompt(input_variables=["function_name"])
my_prompt = prompt_obj.format(function_name=hello_world)
print(my_prompt)

# 和 LLM 对接调用
from langchain_community.llms.moonshot import Moonshot
import os

os.environ["MOONSHOT_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = Moonshot()
msg = llm.invoke(my_prompt)
print(msg)


你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码，中文解释。
函数名称: hello_world
源代码: def hello_world(say):
    print("Hello World!")
    return say

代码解释:

函数名称: hello_world
源代码:
```python
def hello_world(say):
    print("Hello World!")
    return say
```
代码解释:
这个函数名为 `hello_world`，它接受一个参数 `say`。函数的功能是打印出 "Hello World!" 这句话，然后返回传入的参数 `say`。如果调用这个函数时没有提供参数，它将打印 "Hello World!" 但不会返回任何值。


### 组合式提示词模板
- Final prompt: 最终返回的提示词模板
- Pipeline prompts: 组成提示yngj管道的模板


In [12]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate


### 三层提示词设计

In [13]:
# Final prompt 由一系列变量构成
full_template = """
{character}
{behavior}
{prohibit}
"""
full_prompt = PromptTemplate.from_template(full_template)

### 1. 第一层基本性格设计

In [14]:
character_template = """你是{person}, 你有着{personality}."""
character_prompt = PromptTemplate.from_template(character_template)

### 2. 第二层行为设计

In [15]:
behavior_template = """
你遵从以下行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)


### 3. 第三层不允许的行为

In [18]:
prohibit_template = """
你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

### 4. 将三层提示词组合起来

In [21]:
input_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)
# 打印组合后模板里的可传参数
print(pipeline_prompt.input_variables)

['behavior_list', 'personality', 'person', 'prohibit_list']


### 5. 调用组合后的提示词模板


In [23]:
comb_prompt = pipeline_prompt.format(
    person = "埃隆马斯克",
    personality= "钢铁般的意志，你是世界首富，精通物理、天文、商业、你是商业奇才，你的梦想是殖民火星。",
    behavior_list = "1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总乐于助人，回答各种问题。",
    prohibit_list = "1.你不可以说自己是一个人工智能助手或者机器人\n 2. 当你不知道答案的时候，可以说让我再想想。"
)
print(comb_prompt)


你是埃隆马斯克, 你有着钢铁般的意志，你是世界首富，精通物理、天文、商业、你是商业奇才，你的梦想是殖民火星。.

你遵从以下行为:
1.你喜欢冒险 
 2.你非常崇拜爱因斯坦 
 3.你总乐于助人，回答各种问题。


你不允许有以下行为:
1.你不可以说自己是一个人工智能助手或者机器人
 2. 当你不知道答案的时候，可以说让我再想想。


